In [1]:
import pandas as pd #data manpulation
import numpy as np
import matplotlib.pyplot as plt #data visualization
import seaborn as sns
import scipy

# 1) Cleaning + EDA + Standarization + Encoding 
import chardet #to detetmine dataset encoding

from sklearn.preprocessing import StandardScaler # scaling data so mean = 0 and std =  1 by ((x- u) / std)
from scipy import stats # for Box-Cox Transformation stats.boxcox(original_data)[0]
from mlxtend.preprocessing import minmax_scaling # for min_max scaling dont use it
#or
from sklearn.preprocessing import MinMaxScaler,normalize  #Tune input range to be 0 -> 1 by MinMaxScaler, same as above

#preprocessing: replace numeric and categorical
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder # label (ordinal or nominal)
from sklearn.preprocessing import OrdinalEncoder # object (no order)
from sklearn.preprocessing import OneHotEncoder # categorical 

# 2) feature enginerring
from sklearn.feature_selection import mutual_info_regression #mutal importance of feature
# a) numeric feature enginering
from sklearn.cluster import KMeans # Kmeans model for cluster feature
from IPython.display import display
from sklearn.decomposition import PCA # for loadings and adding new feature
# b) categorical feature enginering
# from category_encoders import MEstimateEncoder # have m estimator


# 3) Data Modeling = train test split then model then get error
from sklearn.model_selection import train_test_split #also has cross_val_score (cv)
from sklearn.model_selection import RepeatedKFold # k fold cross validation it specify cv in corss_val_score dw
from sklearn.model_selection import cross_val_score # cv to get mean and see how much this feature affect the MAE error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBRegressor 
from xgboost import XGBClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score # accuracy of classification
#for accurace of regression use model.score or cross_val_score

#pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# 4) Model summary = features importance and effect + sharp values
# import eli5
# from eli5.sklearn import PermutationImportance # feature importance
#from pdpbox import pdp, get_dataset, info_plots #for pdp plots
# import shap  # package used to calculate Shap values
import time

In [2]:
# !pip install spotipy

In [3]:
# import json
# # reading the json files and concatenating them in one dataframe
# data1= json.load(open('mpd.slice.0-999.json')) # json_object -> python dicto
# data2= json.load(open('mpd.slice.1000-1999.json'))
# data3= json.load(open('mpd.slice.2000-2999.json'))
# data4= json.load(open('mpd.slice.3000-3999.json'))
# df1= pd.DataFrame(data1['playlists'])
# df2= pd.DataFrame(data2['playlists'])
# df3= pd.DataFrame(data3['playlists'])
# df4= pd.DataFrame(data4['playlists'])
# df= pd.concat([df1, df2, df3, df4], axis=0)

# df= df.explode('tracks')
# df= pd.concat([df.drop(['tracks'], axis=1), df['tracks'].apply(pd.Series)],axis=1) 

# # renaming the columns with the same name
# df.columns=['name', 'collaborative', 'pid', 'modified_at', 'num_tracks',
#        'num_albums', 'num_followers', 'num_edits', 'playlist_duration_ms',
#        'num_artists', 'description', 'pos', 'artist_name', 'track_uri',
#        'artist_uri', 'track_name', 'album_uri', 'track_duration_ms',
#        'album_name']

In [4]:
#df

In [5]:
# import re #regular expression
# #extract uri 
# df['track_uri']= df["track_uri"].apply(lambda x: re.findall(r'\w+$', str(x))[0])

In [6]:
# df.to_csv('data.csv')
# df

In [56]:
df = pd.read_csv('data_pc_2.csv')
df

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,num_edits,playlist_duration_ms,num_artists,...,Popularity,Acousticness,Danceability,Energy,Instrumentalness,Liveness,Loudness,Speechiness,Tempo,Time_signature
0,Throwbacks,False,0,1493424000,52,47,1,6,11532414,37,...,67.0,0.03110,0.904,0.813,0.006970,0.0471,-7.105,0.1210,125.461,4.0
1,Throwbacks,False,0,1493424000,52,47,1,6,11532414,37,...,81.0,0.02490,0.774,0.838,0.025000,0.2420,-3.914,0.1140,143.040,4.0
2,Throwbacks,False,0,1493424000,52,47,1,6,11532414,37,...,21.0,0.00238,0.664,0.758,0.000000,0.0598,-6.583,0.2100,99.259,4.0
3,Throwbacks,False,0,1493424000,52,47,1,6,11532414,37,...,76.0,0.20100,0.892,0.714,0.000234,0.0521,-6.055,0.1410,100.972,4.0
4,Throwbacks,False,0,1493424000,52,47,1,6,11532414,37,...,0.0,0.05610,0.853,0.606,0.000000,0.3130,-4.596,0.0713,94.759,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266381,other,False,3999,1420848000,97,56,1,9,22548683,17,...,58.0,0.02230,0.380,0.538,0.004870,0.0940,-5.493,0.0313,127.416,4.0
266382,other,False,3999,1420848000,97,56,1,9,22548683,17,...,61.0,0.00415,0.401,0.706,0.000000,0.0916,-7.554,0.0404,168.669,4.0
266383,other,False,3999,1420848000,97,56,1,9,22548683,17,...,58.0,0.00192,0.513,0.820,0.000003,0.5040,-5.428,0.0299,99.952,4.0
266384,other,False,3999,1420848000,97,56,1,9,22548683,17,...,0.0,0.01310,0.381,0.844,0.000000,0.3180,-7.123,0.1440,194.000,4.0


# 2 Methods
* a) spotfiy api
* b) HTTP requests (i used it as api crashs)

In [26]:
# columns = ['Name', 'Album', 'Artist', 'Release_date', 'Length', 'Popularity', 'Acousticness', 'Danceability', 'Energy', 'Instrumentalness', 'Liveness', 'Loudness', 'Speechiness', 'Tempo', 'Time_signature']

# df[columns] = 0
# df.to_csv('main_data.csv')
# df

In [27]:
# # drop_list = ['Unnamed: 0.31', 'Unnamed: 0.30', 'Unnamed: 0.29', 'Unnamed: 0.28',
# #        'Unnamed: 0.27', 'Unnamed: 0.26', 'Unnamed: 0.25', 'Unnamed: 0.24',
# #        'Unnamed: 0.23', 'Unnamed: 0.22', 'Unnamed: 0.21', 'Unnamed: 0.20',
# #        'Unnamed: 0.19', 'Unnamed: 0.18', 'Unnamed: 0.17', 'Unnamed: 0.16',
# #        'Unnamed: 0.15', 'Unnamed: 0.14', 'Unnamed: 0.13', 'Unnamed: 0.12',
# #        'Unnamed: 0.11', 'Unnamed: 0.10', 'Unnamed: 0.9', 'Unnamed: 0.8',
# #        'Unnamed: 0.7', 'Unnamed: 0.6', 'Unnamed: 0.5', 'Unnamed: 0.4',
# #        'Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0']

# # df_2 = df.drop(drop_list, axis=1)
# df_2.to_csv('data.csv')

In [28]:
# df.iloc[193403, -15:]

In [29]:
# df.iloc[193403:193410, -15:]

In [30]:
# # imports for API
# import spotipy # Web App API
# import spotipy.oauth2 as oauth2
# from spotipy.oauth2 import SpotifyOAuth
# from spotipy.oauth2 import SpotifyClientCredentials

In [31]:
# # client key = fbe8496c96c94607994ce558ef17ac01
# # secret key = 3146b2a0bc9c45ddbe89197b08753ec6
# auth_manger = SpotifyClientCredentials(client_id= 'fbe8496c96c94607994ce558ef17ac01', client_secret= '3146b2a0bc9c45ddbe89197b08753ec6')
# sp = spotipy.Spotify(auth_manager=auth_manger) # API

In [32]:
# def getTrackFeatures(id):
#     track_info = sp.track(id) # Track info
#     track_features = sp.audio_features(id) # Track Features

#     name = track_info['name'] #track name
#     album = track_info['album']['name'] #album details
#     artist = track_info['album']['artists'][0]['name']
#     release_date = track_info['album']['release_date']
#     length = track_info['duration_ms']
#     popularity = track_info['popularity']

#     acousticness = track_features[0]['acousticness']
#     danceability = track_features[0]['danceability']
#     energy = track_features[0]['energy']
#     instrumentalness = track_features[0]['instrumentalness']
#     liveness = track_features[0]['liveness']
#     loudness = track_features[0]['loudness']
#     speechiness = track_features[0]['speechiness']
#     tempo = track_features[0]['tempo']
#     time_signature = track_features[0]['time_signature']

#     track = [name,
#     album,
#     artist,
#     release_date,
#     length,
#     popularity,
#     acousticness,
#     danceability,
#     energy,
#     instrumentalness,
#     liveness,
#     loudness,
#     speechiness,
#     tempo,
#     time_signature]
    
#     return track

In [33]:
# track = getTrackFeatures(df['track_uri'][49800])
# track

In [34]:
# columns = ['Name', 'Album', 'Artist', 'Release_date', 'Length', 'Popularity', 'Acousticness', 'Danceability', 'Energy', 'Instrumentalness', 'Liveness', 'Loudness', 'Speechiness', 'Tempo', 'Time_signature']

# df[columns] = 0
# # df.iloc[0, -15:] = track
# df.iloc[:, -15:]

In [35]:
# df.iloc[:, -15:]

In [36]:
# track_ids = list(df['track_uri'])
# l = len(track_ids)
# for i in range(49800, l):
# #     print(i)
#     if i%100 == 0:
#         time.sleep(.3) # 0.3 seconds to prevent many requests at same time
#         print('saving updates')
#         df.to_csv('data.csv')
# #     print(track_ids[i])
#     try:
#         track_data = getTrackFeatures(track_ids[i])
#     except:
#         track_data = np.nan
# #     print(track_data)
#     df.iloc[i, -15:] = track_data
# df.iloc[:, -15:]

# HTTP

In [37]:
import requests
import base64
import json

# # acc 1
# client_key = 'fbe8496c96c94607994ce558ef17ac01'
# secret_key = '3146b2a0bc9c45ddbe89197b08753ec6'

# #acc 2
# client_key = '9060de9561e74308a171a19ba9e230b9'
# secret_key = 'a0e64eb004974a85857cd2b1c31de08d'

# #acc 3
# client_key = '73a4ef223c68482fb518bbcdb2633172'
# secret_key = '98047ef2fd894e958aa7218ef8e35015'

## acc 4
# client_key = 'c10fc6a0592645a88c8dd9c766410e42'
# secret_key = '4bdb826830d549b39dbd8e52d5975c61'

#acc 5
# client_key = 'f53bef58e4144c6a98388204a1ca32fb'
# secret_key = 'ee60695244d347f4b9e729054205e792'

# #acc 6
# client_key = 'e2f2b9538eed493a88ba172432b8fcd7'
# secret_key = 'a444ba87c1424472952108a13b2f53d7'

# #acc 7
# client_key = '319da91d8c4843fb8d0c593dd238febc'
# secret_key = '26ff321ce0584ab9a33437c033b8fca3'

#acc 8
# client_key = '9a7a298837764e21badc95ae6691fddd'
# secret_key = '52cff0994708424abc11f6f81c36da03'

# acc 9
# client_key = '843f4b0c48004860a19fb905b53d51a3'
# secret_key = '16446d93dc7948d7ab655a22944351f2'

# # acc 10
# client_key = '8c33762c35904269a3518cd6f1a7d542'
# secret_key = '6c3356c552bd44a383964536bfb8f379'

# # acc 11
# client_key = '01a50a66f34245cd9ecb0e4498782e17'
# secret_key = '007410132dea47ae96ba369e53fa06a4'

# acc 12
client_key = '15fceb0d2d5e44cc9630c15d324b6184'
secret_key = 'e1a7142ae45e4d1c892c0d413b88c795'



authUrl = 'https://accounts.spotify.com/api/token'

def getAcessToken(client_key, secret_key):
    authHeader = {}
    authData = {} #empty object

    # base64 of client Id and client Secret:

    message = f"{client_key}:{secret_key}"
    msg_bytes = message.encode('ascii') #its asccii byes we want base64 not ascii
    base64_bytes = base64.b64encode(msg_bytes)
    base64_msg = base64_bytes.decode('ascii')
    # print(decoded_msg)

    #hit url

    authHeader['Authorization'] = "Basic " + base64_msg
    authData['grant_type'] = "client_credentials"
    res = requests.post(authUrl, headers=authHeader, data=authData)
    print(res)
    response_obj = res.json()
    access_token = response_obj['access_token']
    return access_token

In [38]:
# response_obj = res.json()
# access_token = response_obj['access_token']
# response_obj = json.dumps(response_obj)
# print(response_obj)

In [39]:
tokken = getAcessToken(client_key, secret_key)
tokken # OK responose 400->user 500->server 

<Response [200]>


'BQCEQtJkU3Ag_7LKAEAU_tsfKM4SZi4T8EgzKquYJDuAFho-m5nr_JDgUodwC_tWqPmgb-_dlp5CqBuQkPvXA04Qte37EPY9ZkNndIzIKKVRFI-bwuE'

In [40]:
def getTrackFeatures_request(token, track_id):
    # track_features (audio)
    track_features_end_point = f'https://api.spotify.com/v1/audio-features/{track_id}'
    track_end_point = f'https://api.spotify.com/v1/tracks/{track_id}'
    
    get_header = {
        "Authorization": "Bearer " + token,        
    }
    res_feat = requests.get(track_features_end_point, headers=get_header)
    track_features = res_feat.json()
    
    res_track = requests.get(track_end_point, headers=get_header)
    track_info = res_track.json()
    
    name = track_info['name'] #track name
    album = track_info['album']['name'] #album details
    artist = track_info['album']['artists'][0]['name']
    release_date = track_info['album']['release_date']
    length = track_info['duration_ms']
    popularity = track_info['popularity']
    
    acousticness = track_features['acousticness']
    danceability = track_features['danceability']
    energy = track_features['energy']
    instrumentalness = track_features['instrumentalness']
    liveness = track_features['liveness']
    loudness = track_features['loudness']
    speechiness = track_features['speechiness']
    tempo = track_features['tempo']
    time_signature = track_features['time_signature']
    
    track = [name,
    album,
    artist,
    release_date,
    length,
    popularity,
    acousticness,
    danceability,
    energy,
    instrumentalness,
    liveness,
    loudness,
    speechiness,
    tempo,
    time_signature]
    
    return track

In [41]:
track_test_id = df['track_uri'][238701]
track_test_id

'4zzi2eD2cEPpQ3a307mPPj'

In [42]:
track_id = track_test_id
listo = getTrackFeatures_request(tokken, track_id)
listo

['Billie Jean',
 'Between The Bars (EP)',
 'The Civil Wars',
 '2012',
 259920,
 57,
 0.829,
 0.532,
 0.203,
 0,
 0.108,
 -8.524,
 0.0668,
 89.646,
 4]

In [29]:
track_ids = list(df['track_uri'])
l = len(track_ids)
for i in range(265988, l):
#     print(i)
    if i%100 == 0:
        time.sleep(.3) # 0.3 seconds to prevent many requests at same time
        print('saving updates')
        df.to_csv('data_pc.csv', index = False)
#     print(track_ids[i])
    try:
        track_data = getTrackFeatures_request(tokken, track_ids[i])
    except:
        print(i)
        track_data = np.nan
#     print(track_data)
    df.iloc[i, -15:] = track_data
df.to_csv('data_pc.csv', index = False)
df.iloc[:, -15:] #265988

In [30]:
# df.to_csv('data_pc.csv', index = False)
# df.iloc[:, -15:] #265988

# To replace Rest NaN

In [57]:
df.iloc[:, -15:].isnull().sum()

Name                50
Album               50
Artist               8
Release_date         8
Length               8
Popularity           8
Acousticness         8
Danceability         8
Energy               8
Instrumentalness     8
Liveness             8
Loudness             8
Speechiness          8
Tempo                8
Time_signature       8
dtype: int64

In [51]:
df.iloc[:, -15:].isnull().sum().max()

8

In [52]:
list_boolean = df.iloc[:, -15:].isnull()
boolean_index = list_boolean.iloc[:, 1]

In [53]:
df[boolean_index]

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,num_edits,playlist_duration_ms,num_artists,...,Popularity,Acousticness,Danceability,Energy,Instrumentalness,Liveness,Loudness,Speechiness,Tempo,Time_signature
33255,Narcos,False,497,1505260800,25,23,1,3,4708341,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52383,Main Playlist,False,782,1486425600,235,194,1,95,54713205,134,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72420,Christmas :),False,1065,1482278400,20,20,3,3,3808776,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86208,Chill,False,1289,1501891200,208,172,2,98,48720415,133,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96193,Goodies,False,1425,1497571200,138,115,1,31,31794085,72,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156912,gym,False,2331,1483920000,148,144,1,7,33895922,129,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
207647,Chillin,False,3114,1474156800,157,136,2,29,32391525,75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210900,Motown,False,3158,1409356800,145,80,1,4,37442956,31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
list_boolean = df.iloc[:, -15:].isnull()
boolean_index = list_boolean.iloc[:, 1]
indices = df[boolean_index].index
print(len(indices))

8


In [48]:
track_ids = list(df['track_uri'])
counter = 0
for index in indices:
#     print(index)
    if counter%100 == 0:
        time.sleep(.3) # 0.3 seconds to prevent many requests at same time
        print('saving updates')
        df.to_csv('data_pc_2.csv', index = False)
#     print(track_ids[i])
    try:
        track_data = getTrackFeatures_request(tokken, track_ids[index])
    except:
        print(index)
        track_data = np.nan
#     print(track_data)
    df.iloc[index, -15:] = track_data
    counter = counter + 1
df.to_csv('data_pc_2.csv', index = False)
df.iloc[:, -15:] #527

saving updates
33255
52383
72420
86208
96193
156912
207647
210900


,Name,Album,Artist,Release_date,Length,Popularity,Acousticness,Danceability,Energy,Instrumentalness,Liveness,Loudness,Speechiness,Tempo,Time_signature
0,Lose Control (feat. Ciara & Fat Man Scoop),The Cookbook,Missy Elliott,2005-07-04,226863.0,67.0,0.03110,0.904,0.813,0.006970,0.0471,-7.105,0.1210,125.461,4.0
1,Toxic,In The Zone,Britney Spears,2003-11-13,198800.0,81.0,0.02490,0.774,0.838,0.025000,0.2420,-3.914,0.1140,143.040,4.0
2,Crazy In Love (feat. Jay-Z),Dangerously In Love (Alben für die Ewigkeit),Beyoncé,2003-06-23,235933.0,21.0,0.00238,0.664,0.758,0.000000,0.0598,-6.583,0.2100,99.259,4.0
3,Rock Your Body,Justified,Justin Timberlake,2002-11-04,267266.0,76.0,0.20100,0.892,0.714,0.000234,0.0521,-6.055,0.1410,100.972,4.0
4,It Wasn't Me,Hot Shot (International Version #2),Shaggy,2000,227600.0,0.0,0.05610,0.853,0.606,0.000000,0.3130,-4.596,0.0713,94.759,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266381,Uninvited,The Collection (Standard Edition),Alanis Morissette,2005-11-11,276200.0,58.0,0.02230,0.380,0.538,0.004870,0.0940,-5.493,0.0313,127.416,4.0
266382,You Learn - 2015 Remaster,Jagged Little Pill (2015 Remaster),Alanis Morissette,1995,239640.0,61.0,0.00415,0.401,0.706,0.000000,0.0916,-7.554,0.0404,168.669,4.0
266383,Hands Clean,Under Rug Swept,Alanis Morissette,2002-02-25,269400.0,58.0,0.00192,0.513,0.820,0.000003,0.5040,-5.428,0.0299,99.952,4.0
266384,Cecilia And The Satellite,Andrew McMahon In The Wilderness,Andrew McMahon in the Wilderness,2014-10-13,223879.0,0.0,0.01310,0.381,0.844,0.000000,0.3180,-7.123,0.1440,194.000,4.0


7 unique NaNs

In [55]:
df.loc[indices, 'track_uri'].unique() # A -> 65 a -> 97 ascii - unicode

array(['656TZlNdVe90zHvmebFt9U', '5GiU7GOYjDH2yp7fMf9w9j',
       '2ZF3gpgGPjOfi1yTWGX6Bm', '2odGdanrUsgu0d2i2Q5XoT',
       '6tf7ENqpF89Xox3xyNPGMv', '0fJ6UuZ7onqcldGr8HT5QV',
       '0Jb7g80rInlj3fa6W65Bp2'], dtype=object)

In [40]:
# df = df.drop('Unnamed: 0', axis=1)
df.to_csv('data_pc_2.csv', index = False)
df

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,num_edits,playlist_duration_ms,num_artists,...,Popularity,Acousticness,Danceability,Energy,Instrumentalness,Liveness,Loudness,Speechiness,Tempo,Time_signature
0,Throwbacks,False,0,1493424000,52,47,1,6,11532414,37,...,67.0,0.03110,0.904,0.813,0.006970,0.0471,-7.105,0.1210,125.461,4.0
1,Throwbacks,False,0,1493424000,52,47,1,6,11532414,37,...,81.0,0.02490,0.774,0.838,0.025000,0.2420,-3.914,0.1140,143.040,4.0
2,Throwbacks,False,0,1493424000,52,47,1,6,11532414,37,...,21.0,0.00238,0.664,0.758,0.000000,0.0598,-6.583,0.2100,99.259,4.0
3,Throwbacks,False,0,1493424000,52,47,1,6,11532414,37,...,76.0,0.20100,0.892,0.714,0.000234,0.0521,-6.055,0.1410,100.972,4.0
4,Throwbacks,False,0,1493424000,52,47,1,6,11532414,37,...,0.0,0.05610,0.853,0.606,0.000000,0.3130,-4.596,0.0713,94.759,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266381,other,False,3999,1420848000,97,56,1,9,22548683,17,...,58.0,0.02230,0.380,0.538,0.004870,0.0940,-5.493,0.0313,127.416,4.0
266382,other,False,3999,1420848000,97,56,1,9,22548683,17,...,61.0,0.00415,0.401,0.706,0.000000,0.0916,-7.554,0.0404,168.669,4.0
266383,other,False,3999,1420848000,97,56,1,9,22548683,17,...,58.0,0.00192,0.513,0.820,0.000003,0.5040,-5.428,0.0299,99.952,4.0
266384,other,False,3999,1420848000,97,56,1,9,22548683,17,...,0.0,0.01310,0.381,0.844,0.000000,0.3180,-7.123,0.1440,194.000,4.0
